In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from collections import deque

In [2]:
# Implementation of the WEISS 1997 PCR coalescent model
# https://academic.oup.com/nar/article/25/15/3082/2902152/

C = 20 # num PCR cycles
p = 0.6 # PCR efficiency: change with time later
family_size = 10

s = np.zeros(C+1)
s[0] = 1 # start with one molecule

# forward sampling
for i in np.arange(C):
    s[i+1] = s[i] + np.random.binomial(s[i], p)

In [3]:
class Node:
    def __init__(self, cycle, index, leaf = False):
        self.cycle = cycle
        self.index = index
        self.leaf = leaf
        self.newly_synthesized = False
        self.coalescent_as = []
        self.children = []
        self.parent = None
        self.error_source_idx = None
        self.error_source_cycle = None
        self.error_type = ""

        if leaf:
            self.left = -1
            self.right = -1
        self.mutation = []
    
    def get_cycle(self):
        return self.cycle
    
    def get_index(self):
        return self.index
    
    def set_newly_synthesized(self):
        self.newly_synthesized = True
    
    def set_parent(self, node):
        self.parent = node
    
    def get_parent(self):
        return self.parent
    
    def add_child(self, node):
        self.children.append(node)
        
    def get_children(self):
        return self.children
    
    def set_coalescent_as(self, role):
        self.coalescent_as.append(role)
    
    def get_coalescent(self):
        return self.coalescent_as

    def set_error(self, source_cycle, source_idx, source=False):
        if self.cycle == C:
            self.error_source_idx = source_idx
            self.error_source_cycle = source_cycle
            self.error_type = "source" if source else "inherited"
            return

        else:   
            if source:
                assert source_cycle == self.cycle, "source_cycle = " + str(source_cycle) + ", cycle = " + str(self.cycle)
                assert source_idx == self.index, "source_idx = " + str(source_idx) + ", index = " + str(self.index)
                self.error_source_cycle = source_cycle
                self.error_source_idx = source_idx
                self.error_type = "source"
                for child in self.get_children():
                    child.set_error(source_cycle, source_idx)
            else:
                assert source_cycle < self.cycle, "source_cycle = " + str(source_cycle) + ", cycle = " + str(self.cycle) 
                self.error_source_cycle = source_cycle
                self.error_source_idx = source_idx
                self.error_type = "inherited"
                for child in self.get_children():
                    child.set_error(source_cycle, source_idx)
    
    def print_error(self):
        if self.error_type != "":
            print("source cycle = " + str(self.error_source_cycle) + ", source index = " + str(self.error_source_idx) + ", type = " + self.error_type)

In [16]:
# trace the genealogy of n duplicate reads
n = np.zeros(C+1) # number of ancestors of observed reads at cycle i
r = np.zeros(C+1) # number of newly synthesized ancestors of observed reads at cycle i
l = np.zeros(C+1) # number of coalescent events at cycle i

n[-1] = family_size
tree = deque()
tree.appendleft([ Node(cycle=C, index=j, leaf=True) for j in np.arange(family_size) ])

for i in np.arange(C)[::-1] + 1:
    # sample the number of newly synthesized molecules within the sampled population
    r[i] = np.random.hypergeometric(s[i] - s[i-1], s[i-1], n[i]) # hypergeometric(good balls, bad balls, num draws)
    if r[i] == 0:
        # no newly synthesized moelcules among the sampled molecules, so no coalescent
        l[i] = 0
        n[i-1] = n[i]
        
        nodes_i_minus_1 = [ Node(cycle=i-1, index=j) for j in np.arange(n[i-1]) ]
        tree.appendleft(nodes_i_minus_1)
        continue
    # sample the number of coalescents at cycle i
    l[i] = np.random.hypergeometric(n[i]-r[i], s[i-1] - (n[i]-r[i]), r[i])
    n[i-1] = n[i] - l[i]
    
    # pop nodes at cycle i and create them for the previous generation i-1
    nodes_i = tree.popleft()
    nodes_i_minus_1 = [ Node(cycle=i-1, index=j) for j in np.arange(n[i-1]) ]
    
    # assign newly synthesized molecules
    newly_synthesized_indices = np.random.choice(np.arange(len(nodes_i)), replace=False, size=int(r[i]))
    existing_molecule_indices = np.setdiff1d(np.arange(len(nodes_i)), newly_synthesized_indices)
    
    for idx in newly_synthesized_indices:
        nodes_i[idx].set_newly_synthesized()

    # choose a pairs of newly synthesized and exisiting molecules that originate from the same existing molecule
    coalescing_exisiting_indices = np.random.choice(newly_synthesized_indices, replace=False, size=int(l[i]))
    coalescing_newly_synthesized_indices = np.random.choice(existing_molecule_indices, replace=False, size=int(l[i]))
       
    # choose parents randomly from the i-1th generation
    parents_of_coalescing_nodes_indices = np.random.choice(np.arange(n[i-1]), replace=False, size=int(l[i]))
    
    non_coalescing_node_indices_i = np.setdiff1d(np.arange(len(nodes_i)), np.union1d(coalescing_exisiting_indices, coalescing_newly_synthesized_indices))
    non_coalescing_node_indices_i_minus_1 = np.setdiff1d(np.arange(len(nodes_i_minus_1)), parents_of_coalescing_nodes_indices)
    assert len(non_coalescing_node_indices_i) == len(non_coalescing_node_indices_i_minus_1)
    
    for (parent_idx, new_idx, existing_idx) in zip(parents_of_coalescing_nodes_indices, coalescing_exisiting_indices,coalescing_newly_synthesized_indices):
        nodes_i_minus_1[int(parent_idx)].add_child(nodes_i[int(new_idx)])
        nodes_i_minus_1[int(parent_idx)].add_child(nodes_i[int(existing_idx)])

        nodes_i_minus_1[int(parent_idx)].set_coalescent_as("parent")
        nodes_i[int(new_idx)].set_coalescent_as("child")
        nodes_i[int(existing_idx)].set_coalescent_as("child")
    
    # randomly pair up the remaining nodes
    for parent_idx, child_idx in zip(non_coalescing_node_indices_i_minus_1, non_coalescing_node_indices_i):
        nodes_i_minus_1[int(parent_idx)].add_child(nodes_i[int(child_idx)])
    
    tree.appendleft(nodes_i)
    tree.appendleft(nodes_i_minus_1)

In [17]:
# add mutations
u = 1e-2 # per generation PCR error rate, in the units of mutations per generation
error_locs = []
for cycle in np.arange(C):
    num_edges = n[cycle+1]
    errors = np.random.poisson(u, int(num_edges))
    if np.sum(errors) > 0:
        for err_node_idx in np.argwhere(errors).flatten():
            list(tree)[cycle+1][err_node_idx].set_error(cycle+1, err_node_idx, True)
            error_locs.append((cycle+1, err_node_idx))
error_locs

[(6, 3), (8, 6), (14, 6), (16, 2)]

In [18]:
for i, nodes in enumerate(tree):
    print("cycle = " + str(i))
    print("num nodes = " + str(len(nodes)))
    for node in nodes:
        print("index = " + str(node.index) + ", num children = " + str(len(node.get_children())) + ", coalescent = " + str(node.get_coalescent()))
        node.print_error()

cycle = 0
num nodes = 1
index = 0.0, num children = 0, coalescent = []
cycle = 1
num nodes = 1
index = 0.0, num children = 0, coalescent = []
cycle = 2
num nodes = 1
index = 0.0, num children = 2, coalescent = ['parent']
cycle = 3
num nodes = 2
index = 0.0, num children = 2, coalescent = ['parent', 'child']
index = 1.0, num children = 2, coalescent = ['parent', 'child']
cycle = 4
num nodes = 4
index = 0.0, num children = 1, coalescent = ['child']
index = 1.0, num children = 2, coalescent = ['parent', 'child']
index = 2.0, num children = 2, coalescent = ['parent', 'child']
index = 3.0, num children = 1, coalescent = ['child']
cycle = 5
num nodes = 6
index = 0.0, num children = 1, coalescent = ['child']
index = 1.0, num children = 1, coalescent = ['child']
index = 2.0, num children = 1, coalescent = ['child']
index = 3.0, num children = 1, coalescent = []
index = 4.0, num children = 1, coalescent = ['child']
index = 5.0, num children = 1, coalescent = []
cycle = 6
num nodes = 6
index = 0

In [62]:
str(len(node.get_children()))

'0'

In [19]:
zip([3,4],[5,6],[7,8])

[(3, 5, 7), (4, 6, 8)]

In [8]:
trees.appendleft(Node(cycle=C, index=j, leaf=False) for j in np.arange(family_size))

NameError: name 'trees' is not defined

In [9]:
a = deque()
a.appendleft(3)
a.appendleft(5)
a

deque([5, 3])

In [10]:
a.popleft()

5

In [11]:
d = deque()
d.append(3)
d.appendleft(13)
d.append(140)


[(9, 1), (13, 1)]

In [59]:
# np.random.poisson(u, num_edges)

START HERE GET RESULTS (QUAL) and also be smarter about where in the tree mutaions occur. i.e. keep track of them.

1.0